# Evaluation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from src import preprocess as prep
from src.evaluation import evaluate_segments
import random
from src.Video import Video
from src.Segment import Segment
from src import search
import cv2

## Parameters

In [55]:
NUM_VIDEOS = 20
GRID_SIZE = 2
BINS = [180, 180]
HIST_FRAME_SKIP = 20
REFRESH = True

Loading / processing dataset...
Done processing!TRAINING SET:
Num. videos:   20
Num. segments: 2198
Duration:      7,556.4 s
Num frames:      210264
Num histograms:      11702


## Load training set

In [416]:
training_set = prep.load_training_set(range(1, NUM_VIDEOS+1), GRID_SIZE, BINS, HIST_FRAME_SKIP, force_refresh=REFRESH)

Loading / processing dataset...
Done processing!

In [417]:
# Print statistics
print("TRAINING SET:")
print("Num. videos:   {:d}".format( len(training_set)) )
print("Num. segments: {:d}".format( np.sum([len(video.segments) for video in training_set])) )
print("Duration:      {:,.1f} s".format( np.sum([np.sum([segment.duration() for segment in video.segments]) for video in training_set])) )
print("Num frames:      {:d}".format( np.sum([np.sum([segment.num_frames() for segment in video.segments]) for video in training_set])) )
print("Num histograms:      {:d}".format( np.sum([np.sum([len(segment.histograms) for segment in video.segments]) for video in training_set])) )

TRAINING SET:
Num. videos:   2
Num. segments: 189
Duration:      529.7 s
Num frames:      14542
Num histograms:      827


## Select random test set

In [57]:
test_n_segments = 100
test_set = []
labels = []

for i in range(test_n_segments):
    
    # Find random video
    video = random.choice(training_set)
    
    # Select random segment and add histogram to test set
    segment = random.choice(video.segments)
    test_set.append(segment.histograms)
    labels.append(segment)

In [63]:
# Print statistics
print("TEST SET:")
print("Num. histograms: {:d}".format( np.sum([len(histogram) for histogram in test_set]) ))

TEST SET:
Num. histograms: 630


<br>

## Run model on test set

In [78]:
%timeit search.findFrame(test_set[0], 0, training_set, cv2.HISTCMP_CORREL)
%timeit search.findFrame(test_set[0], 0, training_set, cv2.HISTCMP_CHISQR)
%timeit search.findFrame(test_set[0], 0, training_set, cv2.HISTCMP_INTERSECT)
%timeit search.findFrame(test_set[0], 0, training_set, cv2.HISTCMP_BHATTACHARYYA)
%timeit search.findFrame(test_set[0], 0, training_set, cv2.HISTCMP_CHISQR_ALT)
%timeit search.findFrame(test_set[0], 0, training_set, cv2.HISTCMP_KL_DIV)

%timeit search.findFrame(test_set[0], 0, training_set, cv2.HISTCMP_KL_DIV, 5)

# %timeit search.find(test_set[0], training_set, cv2.HISTCMP_CORREL)
# %timeit search.find(test_set[0], training_set, cv2.HISTCMP_CHISQR)
# %timeit search.find(test_set[0], training_set, cv2.HISTCMP_INTERSECT)
# %timeit search.find(test_set[0], training_set, cv2.HISTCMP_BHATTACHARYYA)
# %timeit search.find(test_set[0], training_set, cv2.HISTCMP_CHISQR_ALT)
# %timeit search.find(test_set[0], training_set, cv2.HISTCMP_KL_DIV)

4.36 s ± 31.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.66 s ± 35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.29 s ± 59.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.05 s ± 24.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.12 s ± 26.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.35 s ± 37.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.22 s ± 46.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [67]:
results = []

for i, histograms in enumerate(test_set):
    print('\rSearching segment {}/{} - Frames in segment: {}'.format(i+1, len(test_set), len(histograms)), end='', flush=True)

#     results.append(search.find(histogram, training_set, cv2.HISTCMP_INTERSECT))
    results.append(search.findFrame(histograms,  0, training_set, cv2.HISTCMP_INTERSECT))

Searching segment 100/100 - Frames in segment 2

## Evaluate performance

In [79]:
evaluate_segments(results, labels)

Segment evaluation:
Correct: 96
Wrong:   4
Total:   100
TPR:     96.0%


In [37]:
hists = training_set[0].segments[5].histograms
display(len(hists))
# 0 = altijd perfect match
tf = 1
search.findFrame(hists, tf, training_set, cv2.HISTCMP_INTERSECT, prints=True)


3

best_dist_indices [array([ 45,  62,  31,  68,  72, 109,  90,   5,  93,   0]), array([10, 34, 23, 24,  3, 33,  2, 72, 73, 21])]
video 00001 - segment 5


<Segment[00001.mp4, start=8.0 sec, end=10.0 sec, duration=2.0 sec, num_frames=49]>